In [1]:
#Importe
import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

In [2]:
def generischer_optimierer (zielfunktion, cov_matrix):

    n = cov_matrix.shape[0]
    w0 = 1 / n

    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, 1)] * n

    res = minimize(zielfunktion, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'maxiter': 1000})

    return res.x

In [3]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

In [4]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

In [5]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [6]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

In [7]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

In [8]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [9]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

In [10]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [11]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [12]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [13]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [14]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [15]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

In [16]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [17]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

# Checks S&P500

In [18]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [19]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [20]:
#Ein einzelner Zeitraum (1 Trainingsfenster + anschließendes Testzeitfenster)
#Die Richtigkeit der Methoden soll getestet werden

#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2010-04-01')
end_datum = pd.Timestamp('2011-04-30')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)
    
print(training_anfang)
    
#Definition des Trainingsdatensatzes und des Testdatensatzes
training_daten = returns.loc[training_anfang:training_ende]
test_daten = returns.loc[test_anfang:test_ende]
training_daten = training_daten.dropna(axis=1)
    
#Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
max_null_tage = 63
null_zaehler = (training_daten == 0).sum()
valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
training_daten = training_daten[valide_tickers]
    
#Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
test_daten = test_daten[vorhandene_assets]
test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
#Erzeugung Matrizen
cov_matrix = training_daten.cov()
corr_matrix = training_daten.corr()


2010-04-01 00:00:00


In [21]:
training_daten.info

<bound method DataFrame.info of                    A      AAPL       ABT      ADBE       ADI       ADM  \
Date                                                                     
2010-04-01  0.004652  0.004128  0.005315  0.010461  0.000000  0.007266   
2010-04-05  0.007815  0.010679 -0.000566  0.006995  0.021860 -0.022329   
2010-04-06 -0.007467  0.004403 -0.001700 -0.008058 -0.005433 -0.008082   
2010-04-07 -0.007812  0.004425 -0.006435 -0.017927  0.005804  0.000708   
2010-04-08 -0.008749 -0.002702 -0.004190 -0.003423 -0.015954 -0.006726   
...              ...       ...       ...       ...       ...       ...   
2011-03-25  0.002961  0.019045 -0.001663  0.001534 -0.002046 -0.021347   
2011-03-28 -0.001817 -0.003129  0.001041 -0.006434 -0.001538  0.001416   
2011-03-29  0.013649  0.001484  0.006240  0.005859  0.013860  0.010467   
2011-03-30  0.011894 -0.006639  0.011988  0.010117  0.000000  0.006159   
2011-03-31 -0.006875 -0.000344  0.001838  0.006373 -0.003038  0.001948   

     

In [22]:
#Summe der letzten Zeile
training_daten.iloc[-1].sum()

-0.17347799619488735

In [23]:
##### ÜBERPRÜFUNG ERSTELLUNG TRAINGSZEITFENSTER!

#Start- und Enddatum sind 01.04.2010 und 31.03.2011
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 253
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 321
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: -0,17347799
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [24]:
test_daten.info

<bound method DataFrame.info of                    A      AAPL       ABT      ADBE       ADI       ADM  \
Date                                                                     
2011-04-01  0.008039 -0.011334  0.006524  0.030157 -0.005587  0.013052   
2011-04-04 -0.006867 -0.009781  0.017825  0.006148 -0.016854  0.006579   
2011-04-05  0.011376 -0.006741 -0.005174 -0.005528 -0.003117  0.008170   
2011-04-06 -0.001103 -0.002508  0.013803  0.001755  0.023450 -0.012426   
2011-04-07 -0.009273  0.000118 -0.003749  0.007009  0.001273 -0.002462   
2011-04-08  0.000446 -0.008933  0.000000 -0.009571 -0.006865 -0.032904   
2011-04-11  0.002228 -0.012714  0.008319  0.006735 -0.015873  0.008506   
2011-04-12 -0.001111  0.004837  0.002357 -0.008435 -0.017950 -0.000843   
2011-04-13  0.036716  0.011221 -0.011366  0.003520  0.002649 -0.011255   
2011-04-14  0.000859 -0.011037  0.010902  0.003508 -0.004756 -0.002846   
2011-04-15  0.018658 -0.014921  0.017647  0.005243  0.013008 -0.005993   
2011-0

In [25]:
#Summe der letzten Zeile
test_daten.iloc[-1].sum()

0.1631653361235177

In [26]:
##### ÜBERPRÜFUNG ERSTELLUNG TESTZEITFENSTER!

#Start- und Enddatum sind 01.04.2011 und 29.04.2011
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 20
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 321
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: 0,1631653
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [27]:
def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

In [28]:
cov_matrix

,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
A,0.000464,0.000186,0.000091,0.000234,0.000247,0.000156,0.000151,0.000290,0.000133,0.000106,...,0.000199,0.000259,0.000308,0.000112,0.000140,0.000204,0.000261,0.000177,0.000158,0.000296
AAPL,0.000186,0.000265,0.000056,0.000143,0.000142,0.000110,0.000098,0.000194,0.000086,0.000076,...,0.000137,0.000102,0.000247,0.000074,0.000098,0.000139,0.000182,0.000118,0.000101,0.000203
ABT,0.000091,0.000056,0.000084,0.000062,0.000062,0.000061,0.000052,0.000087,0.000057,0.000055,...,0.000055,0.000069,0.000095,0.000050,0.000053,0.000063,0.000087,0.000057,0.000054,0.000091
ADBE,0.000234,0.000143,0.000062,0.000579,0.000189,0.000091,0.000118,0.000305,0.000103,0.000071,...,0.000158,0.000147,0.000238,0.000090,0.000115,0.000159,0.000212,0.000131,0.000117,0.000272
ADI,0.000247,0.000142,0.000062,0.000189,0.000309,0.000113,0.000125,0.000240,0.000110,0.000084,...,0.000163,0.000201,0.000246,0.000086,0.000111,0.000153,0.000224,0.000142,0.000122,0.000241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRAY,0.000204,0.000139,0.000063,0.000159,0.000153,0.000104,0.000115,0.000219,0.000090,0.000080,...,0.000137,0.000134,0.000213,0.000075,0.000101,0.000216,0.000177,0.000119,0.000147,0.000214
XRX,0.000261,0.000182,0.000087,0.000212,0.000224,0.000138,0.000156,0.000284,0.000131,0.000110,...,0.000188,0.000244,0.000319,0.000107,0.000148,0.000177,0.000436,0.000177,0.000157,0.000323
YUM,0.000177,0.000118,0.000057,0.000131,0.000142,0.000108,0.000103,0.000199,0.000087,0.000078,...,0.000123,0.000097,0.000223,0.000077,0.000093,0.000119,0.000177,0.000191,0.000085,0.000188
ZBH,0.000158,0.000101,0.000054,0.000117,0.000122,0.000082,0.000095,0.000140,0.000081,0.000067,...,0.000113,0.000173,0.000157,0.000066,0.000090,0.000147,0.000157,0.000085,0.000227,0.000178


In [29]:
cov_ledoit = ledoit_shrink (training_daten)
cov_ledoit

,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
A,0.000459,0.000180,0.000088,0.000226,0.000238,0.000151,0.000145,0.000280,0.000128,0.000102,...,0.000192,0.000250,0.000298,0.000108,0.000135,0.000197,0.000252,0.000171,0.000152,0.000286
AAPL,0.000180,0.000266,0.000054,0.000138,0.000137,0.000106,0.000095,0.000187,0.000083,0.000074,...,0.000132,0.000099,0.000238,0.000071,0.000095,0.000135,0.000176,0.000114,0.000098,0.000196
ABT,0.000088,0.000054,0.000092,0.000060,0.000060,0.000059,0.000050,0.000084,0.000055,0.000053,...,0.000053,0.000067,0.000091,0.000048,0.000051,0.000061,0.000084,0.000055,0.000052,0.000088
ADBE,0.000226,0.000138,0.000060,0.000570,0.000183,0.000088,0.000114,0.000295,0.000100,0.000069,...,0.000153,0.000142,0.000230,0.000086,0.000111,0.000154,0.000204,0.000127,0.000113,0.000263
ADI,0.000238,0.000137,0.000060,0.000183,0.000309,0.000109,0.000120,0.000232,0.000107,0.000081,...,0.000157,0.000194,0.000238,0.000083,0.000107,0.000147,0.000216,0.000137,0.000118,0.000233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRAY,0.000197,0.000135,0.000061,0.000154,0.000147,0.000101,0.000111,0.000211,0.000087,0.000077,...,0.000132,0.000129,0.000206,0.000072,0.000097,0.000220,0.000171,0.000115,0.000142,0.000207
XRX,0.000252,0.000176,0.000084,0.000204,0.000216,0.000133,0.000150,0.000274,0.000126,0.000106,...,0.000182,0.000235,0.000308,0.000103,0.000143,0.000171,0.000432,0.000171,0.000151,0.000312
YUM,0.000171,0.000114,0.000055,0.000127,0.000137,0.000104,0.000099,0.000192,0.000084,0.000075,...,0.000119,0.000093,0.000215,0.000074,0.000090,0.000115,0.000171,0.000196,0.000082,0.000182
ZBH,0.000152,0.000098,0.000052,0.000113,0.000118,0.000079,0.000091,0.000135,0.000079,0.000065,...,0.000109,0.000167,0.000152,0.000064,0.000086,0.000142,0.000151,0.000082,0.000231,0.000171


In [30]:
corr_matrix

,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
A,1.000000,0.531546,0.460585,0.451202,0.651812,0.469602,0.646952,0.577341,0.504407,0.456077,...,0.569742,0.270835,0.542227,0.546391,0.535958,0.643363,0.579949,0.595690,0.485667,0.509195
AAPL,0.531546,1.000000,0.378888,0.364847,0.495089,0.438227,0.559150,0.511558,0.430670,0.436000,...,0.518305,0.141882,0.574173,0.478854,0.496270,0.582383,0.537092,0.523223,0.412963,0.462250
ABT,0.460585,0.378888,1.000000,0.281776,0.385010,0.431869,0.526555,0.405958,0.509137,0.555114,...,0.372983,0.169961,0.392426,0.572786,0.478709,0.470754,0.455288,0.453722,0.391204,0.371088
ADBE,0.451202,0.364847,0.281776,1.000000,0.447953,0.244463,0.453676,0.544569,0.350262,0.275032,...,0.405224,0.137639,0.375295,0.391666,0.393393,0.450470,0.421159,0.394520,0.322085,0.420224
ADI,0.651812,0.495089,0.385010,0.447953,1.000000,0.417077,0.657085,0.585400,0.512852,0.443820,...,0.571452,0.257316,0.530275,0.515591,0.517416,0.590038,0.609321,0.584922,0.461863,0.509074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRAY,0.643363,0.582383,0.470754,0.450470,0.590038,0.459105,0.722493,0.638402,0.501947,0.506185,...,0.572388,0.205309,0.549644,0.535292,0.564846,1.000000,0.575583,0.587068,0.662847,0.539828
XRX,0.579949,0.537092,0.455288,0.421159,0.609321,0.426421,0.690549,0.583001,0.512041,0.488662,...,0.554990,0.262850,0.579391,0.537617,0.584954,0.575583,1.000000,0.611740,0.497126,0.573419
YUM,0.595690,0.523223,0.453722,0.394520,0.584922,0.506354,0.688336,0.615961,0.511154,0.524451,...,0.547080,0.157512,0.611197,0.583689,0.553544,0.587068,0.611740,1.000000,0.409326,0.505031
ZBH,0.485667,0.412963,0.391204,0.322085,0.461863,0.350160,0.581541,0.397791,0.441146,0.412770,...,0.462401,0.258907,0.394487,0.463691,0.488604,0.662847,0.497126,0.409326,1.000000,0.436624


In [31]:
#aus der geschrumpften Matrix die Korrelationsmatrix bauen
standardabweichung = np.sqrt(np.diag(cov_ledoit))
corr_ledoit = cov_ledoit / np.outer(standardabweichung, standardabweichung)
corr_ledoit

,A,AAPL,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
A,1.000000,0.514098,0.426713,0.441389,0.632309,0.453176,0.609901,0.564435,0.480278,0.429823,...,0.550996,0.266779,0.531312,0.508509,0.510060,0.619387,0.565544,0.571615,0.468142,0.499111
AAPL,0.514098,1.000000,0.347878,0.353714,0.475972,0.419109,0.522403,0.495641,0.406394,0.407219,...,0.496759,0.138505,0.557574,0.441661,0.468057,0.555655,0.519059,0.497577,0.394495,0.449035
ABT,0.426713,0.347878,1.000000,0.261677,0.354561,0.395641,0.471240,0.376768,0.460212,0.496644,...,0.342429,0.158931,0.365038,0.506057,0.432487,0.430241,0.421478,0.413318,0.357976,0.345303
ADBE,0.441389,0.353714,0.261677,1.000000,0.435588,0.236476,0.428715,0.533668,0.334304,0.259820,...,0.392827,0.135903,0.368619,0.365383,0.375278,0.434719,0.411679,0.379480,0.311204,0.412886
ADI,0.632309,0.475972,0.354561,0.435588,1.000000,0.400080,0.615746,0.568889,0.485397,0.415768,...,0.549341,0.251946,0.516491,0.476974,0.489467,0.564650,0.590631,0.557923,0.442532,0.496006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XRAY,0.619387,0.555655,0.430241,0.434719,0.564650,0.437061,0.671912,0.615698,0.471478,0.470601,...,0.546075,0.199503,0.531303,0.491449,0.530288,1.000000,0.553703,0.555729,0.630296,0.521987
XRX,0.565544,0.519059,0.421478,0.411679,0.590631,0.411186,0.650495,0.569526,0.487169,0.460175,...,0.536312,0.258713,0.567287,0.499955,0.556256,0.553703,1.000000,0.586560,0.478815,0.561627
YUM,0.571615,0.497577,0.413318,0.379480,0.557923,0.480463,0.638052,0.592111,0.478556,0.485987,...,0.520223,0.152557,0.588868,0.534129,0.517978,0.555729,0.586560,1.000000,0.387951,0.486743
ZBH,0.468142,0.394495,0.357976,0.311204,0.442532,0.333756,0.541492,0.384115,0.414877,0.384223,...,0.441685,0.251893,0.381791,0.426235,0.459274,0.630296,0.478815,0.387951,1.000000,0.422712


In [32]:
is_in_range = np.all((corr_matrix >= -1) & (corr_matrix <= 1))

print(is_in_range)

True


In [33]:
tol = 1e-12
is_in_range_ledoit = np.all((corr_ledoit >= -1) & (corr_ledoit <= 1 + tol))

print(is_in_range_ledoit)

True


In [34]:
#ÜBERPRÜFUNG KOVARIANZ_MATRIZEN

#Alle 4 Kovarianz- und Korrelationsmatrizen haben die gleiche Größe (NxN)
#Erreicht: Ja

#Korrelationsmatrizen haben Werte zwischen -1 und 1 und haben 1 auf der Diagonale
#Erreicht: Ja

#Kovarianz und Korrelationsmatrizen unter Ledoit unterscheiden sich von den gewöhnlichen Matrizen
#Erreicht: Ja

In [35]:
gewichte_EW = equally_weighted(training_daten)
gewichte_MV = min_variance_opt(cov_matrix)
gewichte_MD = max_diversification_portfolio(cov_matrix)
gewichte_IV = inverse_volatility_portfolio(cov_matrix)
gewichte_ERC = get_risk_parity_weights(cov_matrix)
gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
gewichte_herc = herc (training_daten)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.449624609961594e-05
            Iterations: 218
            Function evaluations: 70197
            Gradient evaluations: 218
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2299812027009533
            Iterations: 34
            Function evaluations: 10978
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.977959768297004e-10
            Iterations: 51
            Function evaluations: 16422
            Gradient evaluations: 51


In [36]:
print(1/321)
print(gewichte_EW.min())
print(gewichte_EW.sum())

0.003115264797507788
0.003115264797507788
1.0


In [37]:
#ÜBERPRÜFUNG METHODE EQUALLY WEIGHTED

#Minimales Gewicht muss 1/N entsprechen
#Erreicht: Ja

#Gewichte müssen aufsummiert 1 ergeben
#Erreicht: Ja

In [38]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [39]:
#ÜBERPRÜFUNG METHODE MINIMUM-VARIANCE

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [40]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [41]:
#ÜBERPRÜFUNG METHODE MAXIMUM-DIVERSIFICATION

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [42]:
wert = gewichte_IV.min()
print(f"{wert:.12f}")
wert = gewichte_IV.sum()
print(f"{wert:.12f}")

0.001141892038
1.000000000000


In [43]:
#ÜBERPRÜFUNG METHODE INVERSE-VOLATILITY

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [44]:
wert = gewichte_ERC.min()
print(f"{wert:.12f}")
wert = gewichte_ERC.sum()
print(f"{wert:.12f}")

0.001250843904
1.000000000000


In [45]:
#ÜBERPRÜFUNG METHODE EQUAL-RISK-CONTRIBUTION

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [46]:
wert = gewichte_hrp.min()
print(f"{wert:.12f}")
wert = gewichte_hrp.sum()
print(f"{wert:.12f}")

0.000325306152
1.000000000000


In [47]:
#ÜBERPRÜFUNG METHODE HIERARCHICAL RISK PARITY

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [48]:
wert = gewichte_herc.min()
print(f"{wert:.12f}")
wert = gewichte_herc.sum()
print(f"{wert:.12f}")

0.000031160066
1.000000000000


In [49]:
#ÜBERPRÜFUNG METHODE HIERARCHICAL EQUAL RISK CONTRIBUTION

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [50]:
gewichte_MV = min_variance_opt(cov_ledoit)
gewichte_MD = max_diversification_portfolio(cov_ledoit)
gewichte_IV = inverse_volatility_portfolio(cov_ledoit)
gewichte_ERC = get_risk_parity_weights(cov_ledoit)
gewichte_hrp = hrp_main(corr_ledoit, cov_ledoit, training_daten)
gewichte_herc = herc_ledoit (training_daten)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.422188937100494e-05
            Iterations: 187
            Function evaluations: 60214
            Gradient evaluations: 187
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.257811741701639
            Iterations: 36
            Function evaluations: 11624
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.653631455844135e-10
            Iterations: 52
            Function evaluations: 16744
            Gradient evaluations: 52


In [51]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [52]:
#ÜBERPRÜFUNG METHODE MINIMUM-VARIANCE UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [53]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [54]:
#ÜBERPRÜFUNG METHODE MAXIMUM-DIVERSIFICATION UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [55]:
wert = gewichte_IV.min()
print(f"{wert:.12f}")
wert = gewichte_IV.sum()
print(f"{wert:.12f}")

0.001171048667
1.000000000000


In [56]:
#ÜBERPRÜFUNG METHODE INVERSE-VOLATILITY UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [57]:
wert = gewichte_ERC.min()
print(f"{wert:.12f}")
wert = gewichte_ERC.sum()
print(f"{wert:.12f}")

0.001261257925
1.000000000000


In [58]:
#ÜBERPRÜFUNG METHODE EQUAL-RISK-CONTRIBUTION UNTER LEDOIT

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Zielwert des Optimierers unterscheidet sich unter Ledoit von normaler Berechnung
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [59]:
wert = gewichte_hrp.min()
print(f"{wert:.12f}")
wert = gewichte_hrp.sum()
print(f"{wert:.12f}")

0.000397963783
1.000000000000


In [60]:
#ÜBERPRÜFUNG METHODE HRP UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [61]:
wert = gewichte_herc.min()
print(f"{wert:.12f}")
wert = gewichte_herc.sum()
print(f"{wert:.12f}")

0.000005254224
1.000000000000


In [62]:
#ÜBERPRÜFUNG METHODE HERC UNTER LEDOIT

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [63]:
np.savetxt("test_gewichte_EW.csv", gewichte_EW, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_MV.csv", gewichte_MV, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_MD.csv", gewichte_MD, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_IV.csv", gewichte_IV, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_ERC.csv", gewichte_ERC, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_HERC.csv", gewichte_herc, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_HRP.csv", gewichte_hrp, delimiter=",", fmt="%.12f")

In [64]:
#TEST DER ERGEBNISSE (RENDITEN AUF TEST-DATEN ANHAND GEWICHTE)

returns_EW = test_daten @ gewichte_EW
returns_MV = test_daten @ gewichte_MV
returns_MD = test_daten @ gewichte_MD
returns_IV = test_daten @ gewichte_IV
returns_ERC = test_daten @ gewichte_ERC
returns_HRP = test_daten @ gewichte_hrp
returns_HERC = test_daten @ gewichte_herc

print(returns_EW)
print(returns_MV)
print(returns_MD)
print(returns_IV)
print(returns_ERC)
print(returns_HRP)
print(returns_HERC)

Date
2011-04-01    0.006525
2011-04-04    0.000091
2011-04-05    0.000031
2011-04-06    0.002942
2011-04-07   -0.002333
2011-04-08   -0.006014
2011-04-11   -0.003045
2011-04-12   -0.006044
2011-04-13    0.000756
2011-04-14    0.000961
2011-04-15    0.006202
2011-04-18   -0.011288
2011-04-19    0.004675
2011-04-20    0.013318
2011-04-21    0.005277
2011-04-25   -0.001853
2011-04-26    0.009343
2011-04-27    0.005908
2011-04-28    0.002970
2011-04-29    0.000508
dtype: float64
Date
2011-04-01    0.004329
2011-04-04    0.004051
2011-04-05   -0.000739
2011-04-06    0.006092
2011-04-07   -0.001292
2011-04-08   -0.003136
2011-04-11   -0.000228
2011-04-12    0.005429
2011-04-13   -0.000638
2011-04-14    0.003024
2011-04-15    0.006343
2011-04-18   -0.005908
2011-04-19    0.002536
2011-04-20    0.008027
2011-04-21   -0.000486
2011-04-25   -0.007472
2011-04-26    0.008105
2011-04-27    0.006654
2011-04-28    0.003381
2011-04-29    0.003144
dtype: float64
Date
2011-04-01    0.005764
2011-04-04  

In [65]:
#ÜBERPRÜFUNG GEWICHTE UND RETURNS

#Alle Methoden geben 321 Werte (Gewichte) zurück (Array-Längen 321)
#Erreicht: Ja

#Manuell ermittelte Returns stimmen mit der Implementierung überein
#Erreicht: Ja

In [66]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_renditen = pd.read_csv(
    'returnssp500shrinked_neu_f.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [67]:
df_renditen.info

<bound method DataFrame.info of             Date  Equally-Weighted  Minimum-Variance  Maximum-Diversification  \
0     2006-06-01          0.012900          0.010584                 0.010986   
1     2006-06-02          0.001590          0.000545                 0.001427   
2     2006-06-05         -0.019401         -0.011641                -0.015187   
3     2006-06-06         -0.002777          0.002887                -0.002299   
4     2006-06-07         -0.005689         -0.000096                -0.000634   
...          ...               ...               ...                      ...   
4775  2025-05-23         -0.003974          0.001147                -0.000438   
4776  2025-05-27          0.018501          0.009232                 0.008390   
4777  2025-05-28         -0.008744         -0.006262                -0.006523   
4778  2025-05-29          0.003698          0.003004                 0.002230   
4779  2025-05-30         -0.000446          0.004944                 0.006350

In [68]:
#ÜBERPRÜFUNG GESAMMELTE RENDITEN

#Gesammelte Renditen starten am 01.06.2006 (da erstes Training vom 01.06.2005 bis 31.05.2006)
#Erreicht: Ja

#Gesammelte Renditen enden am 30.05.2025 (Ende der Daten)
#Erreicht: Ja

#Gesammelte Renditen sind vollständig (5031 Renditen - 251 Renditen aus erstem Jahr = 4780 berechnete Renditen)
#Erreicht: Ja

#Gesammlte Renditen aus dem Hauptprogramm stimmen mit den hier ermittelten Daten überein (im Testzeitfenster)
#Erreicht: Ja

In [69]:
#Ein einzelner Zeitraum (1 Trainingsfenster + anschließendes Testzeitfenster)
#Die Richtigkeit der Methoden soll getestet werden

#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2022-08-01')
end_datum = pd.Timestamp('2023-08-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)
    
print(training_anfang)
    
#Definition des Trainingsdatensatzes und des Testdatensatzes
training_daten = returns.loc[training_anfang:training_ende]
test_daten = returns.loc[test_anfang:test_ende]
training_daten = training_daten.dropna(axis=1)
    
#Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
max_null_tage = 63
null_zaehler = (training_daten == 0).sum()
valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
training_daten = training_daten[valide_tickers]
    
#Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
test_daten = test_daten[vorhandene_assets]
test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
#Erzeugung Matrizen
cov_matrix = training_daten.cov()
corr_matrix = training_daten.corr()

2022-08-01 00:00:00


In [70]:
print(test_daten["AAP"])

Date
2023-08-01   -0.009007
2023-08-02   -0.055345
2023-08-03    0.034319
2023-08-04   -0.014161
2023-08-07    0.004084
2023-08-08   -0.010379
2023-08-09    0.009070
2023-08-10    0.004635
2023-08-11    0.010485
2023-08-14   -0.016187
2023-08-15   -0.019407
2023-08-16    0.002008
2023-08-17   -0.005868
2023-08-18    0.005327
2023-08-21   -0.023342
2023-08-22   -0.012610
2023-08-23    0.031185
2023-08-24   -0.022033
2023-08-25   -0.056398
2023-08-28    0.000000
2023-08-29    0.000000
2023-08-30    0.000000
2023-08-31    0.000000
Name: AAP, dtype: float64


In [71]:
#Überpüfung Cash-Out

#Ticker AAP ist ab 28.08.2023 nicht mehr auf dem S&P500 gelistet - In Testdaten sollen diese Werte 0 sein
#Erreicht: ja

# JSE

In [72]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [73]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [74]:
#Ein einzelner Zeitraum (1 Trainingsfenster + anschließendes Testzeitfenster)
#Die Richtigkeit der Methoden soll getestet werden

#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2006-07-01')
end_datum = pd.Timestamp('2007-07-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)
    
print(training_anfang)
    
#Definition des Trainingsdatensatzes und des Testdatensatzes
training_daten = returns.loc[training_anfang:training_ende]
test_daten = returns.loc[test_anfang:test_ende]
training_daten = training_daten.dropna(axis=1)
    
#Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
max_null_tage = 63
null_zaehler = (training_daten == 0).sum()
valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
training_daten = training_daten[valide_tickers]
    
#Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
test_daten = test_daten[vorhandene_assets]
test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
#Erzeugung Matrizen
cov_matrix = training_daten.cov()
corr_matrix = training_daten.corr()

2006-07-01 00:00:00


In [75]:
training_daten.info

<bound method DataFrame.info of               ABG.JO    ACL.JO    ADH.JO    AFE.JO    AGL.JO    AIP.JO  \
Date                                                                     
2006-07-03  0.055234  0.015385 -0.022222  0.006838 -0.004161  0.045296   
2006-07-04  0.000567  0.007773 -0.022727  0.018676 -0.003425  0.027333   
2006-07-05 -0.032672 -0.006145  0.000000 -0.007333 -0.017182 -0.020766   
2006-07-06 -0.014057  0.049724 -0.009302 -0.006212  0.019231  0.020543   
2006-07-07  0.016238 -0.018797  0.000000 -0.010813  0.020583  0.019416   
...              ...       ...       ...       ...       ...       ...   
2007-06-25 -0.014159 -0.009901  0.000000  0.000000 -0.015457  0.000000   
2007-06-26 -0.004966 -0.002692 -0.002128 -0.023056 -0.000493 -0.013624   
2007-06-27 -0.009520 -0.020440 -0.029851 -0.018663 -0.025593  0.027569   
2007-06-28  0.015425  0.003858  0.000000  0.004295 -0.004843 -0.018764   
2007-06-29  0.003817 -0.000706  0.010989 -0.004887 -0.097690 -0.005479   

     

In [76]:
#Summe der letzten Zeile
training_daten.iloc[-1].sum()

0.31746404997638666

In [77]:
##### ÜBERPRÜFUNG ERSTELLUNG TRAINGSZEITFENSTER!

#Start- und Enddatum sind 03.07.2006 und 29.06.2007
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 257
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 73
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: 0,31746405
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [78]:
test_daten.info

<bound method DataFrame.info of               ABG.JO    ACL.JO    ADH.JO    AFE.JO    AGL.JO    AIP.JO  \
Date                                                                     
2007-07-02  0.000760 -0.007064  0.010870 -0.005648  0.000000  0.008264   
2007-07-03  0.004939  0.007905 -0.019355  0.000123  0.000000  0.005464   
2007-07-04 -0.017013 -0.011922 -0.057017 -0.000494 -0.090866  0.022283   
2007-07-05  0.000000 -0.007700  0.011628  0.000494  0.011818 -0.003190   
2007-07-06  0.019231  0.001520  0.036782  0.012346  0.012832  0.013333   
2007-07-09  0.037736  0.030272  0.017738  0.018293  0.019169 -0.002632   
2007-07-10  0.021818 -0.023180 -0.010893 -0.005988 -0.027375 -0.000792   
2007-07-11 -0.010463 -0.009524 -0.030837  0.005904  0.022075  0.003433   
2007-07-12 -0.007265 -0.002404 -0.022727  0.005989  0.006480 -0.009053   
2007-07-13 -0.014636  0.012048  0.000000  0.006072  0.009657 -0.004143   
2007-07-16 -0.007353  0.000000  0.023256 -0.008876 -0.011690  0.016000   
2007-0

In [79]:
#Summe der letzten Zeile
test_daten.iloc[-1].sum()

0.6450064482609412

In [80]:
##### ÜBERPRÜFUNG ERSTELLUNG TESTZEITFENSTER!

#Start- und Enddatum sind 02.07.2007 und 31.07.2007
#Erreicht: Ja

#Zielwert Zeilen laut manueller Berechnung in Excel: 22
#Erreicht: Ja 

#Zielwert Spalten laut manueller Berechnung in Excel: 73
#Erreicht: Ja

#Summe der letzten Zeile als Überprüfungswert in Excel: 0,645006448
#Erreicht: Ja

#Berechnung Trainings-Zeitfenster in Ordnung

In [81]:
cov_matrix

,ABG.JO,ACL.JO,ADH.JO,AFE.JO,AGL.JO,AIP.JO,ANG.JO,APN.JO,ARI.JO,ARL.JO,...,TFG.JO,TKG.JO,TON.JO,TRU.JO,TSG.JO,VAL.JO,VOD.JO,WBO.JO,WEZ.JO,WHL.JO
ABG.JO,2.426602e-04,0.000094,0.000048,0.000033,0.000066,8.940100e-05,0.000053,0.000088,0.000091,0.000047,...,0.000073,0.000056,0.000071,0.000084,0.000049,0.000121,0.000056,0.000035,-5.811100e-07,0.000092
ACL.JO,9.431789e-05,0.000278,-0.000020,0.000022,0.000078,6.912870e-05,0.000096,0.000080,0.000052,0.000029,...,0.000072,0.000058,0.000062,0.000057,0.000044,0.000113,0.000058,0.000019,3.844165e-05,0.000065
ADH.JO,4.775602e-05,-0.000020,0.000507,0.000006,0.000065,-7.539678e-07,0.000069,0.000017,0.000014,0.000068,...,0.000028,0.000048,0.000053,0.000051,0.000030,0.000076,0.000048,0.000066,8.302020e-05,0.000012
AFE.JO,3.336598e-05,0.000022,0.000006,0.000227,0.000045,5.822693e-05,0.000027,0.000036,0.000111,0.000035,...,0.000025,0.000020,0.000028,0.000029,0.000028,0.000058,0.000020,0.000024,-3.305285e-05,0.000054
AGL.JO,6.640796e-05,0.000078,0.000065,0.000045,0.000318,5.913965e-05,0.000148,0.000045,0.000090,0.000047,...,0.000049,0.000048,0.000067,0.000032,0.000053,0.000196,0.000048,0.000032,-3.408191e-05,0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAL.JO,1.205948e-04,0.000113,0.000076,0.000058,0.000196,8.641948e-05,0.000189,0.000073,0.000149,0.000050,...,0.000079,0.000066,0.000101,0.000116,0.000082,0.000512,0.000066,0.000030,-7.513572e-05,0.000112
VOD.JO,5.600612e-05,0.000058,0.000048,0.000020,0.000048,3.196925e-05,0.000048,0.000020,0.000042,0.000029,...,0.000016,0.000232,0.000030,0.000007,0.000020,0.000066,0.000232,0.000032,-5.010476e-05,0.000041
WBO.JO,3.462137e-05,0.000019,0.000066,0.000024,0.000032,2.890889e-05,0.000031,0.000014,0.000049,0.000036,...,0.000048,0.000032,0.000027,0.000022,0.000002,0.000030,0.000032,0.000175,7.458540e-06,0.000033
WEZ.JO,-5.811100e-07,0.000038,0.000083,-0.000033,-0.000034,-1.923527e-05,0.000043,0.000056,-0.000015,0.000067,...,0.000058,-0.000050,-0.000019,0.000020,0.000047,-0.000075,-0.000050,0.000007,1.617305e-03,0.000002


In [82]:
cov_ledoit = ledoit_shrink (training_daten)
cov_ledoit

,ABG.JO,ACL.JO,ADH.JO,AFE.JO,AGL.JO,AIP.JO,ANG.JO,APN.JO,ARI.JO,ARL.JO,...,TFG.JO,TKG.JO,TON.JO,TRU.JO,TSG.JO,VAL.JO,VOD.JO,WBO.JO,WEZ.JO,WHL.JO
ABG.JO,2.765819e-04,0.000067,0.000034,0.000024,0.000047,6.368313e-05,0.000038,0.000062,0.000065,0.000033,...,0.000052,0.000040,0.000051,0.000060,0.000035,0.000086,0.000040,0.000025,-4.139429e-07,0.000065
ACL.JO,6.718559e-05,0.000302,-0.000014,0.000016,0.000055,4.924254e-05,0.000068,0.000057,0.000037,0.000021,...,0.000051,0.000041,0.000044,0.000041,0.000032,0.000080,0.000041,0.000014,2.738319e-05,0.000046
ADH.JO,3.401811e-05,-0.000014,0.000465,0.000004,0.000047,-5.370749e-07,0.000049,0.000012,0.000010,0.000049,...,0.000020,0.000034,0.000038,0.000036,0.000022,0.000054,0.000034,0.000047,5.913789e-05,0.000008
AFE.JO,2.376763e-05,0.000016,0.000004,0.000265,0.000032,4.147687e-05,0.000019,0.000026,0.000079,0.000025,...,0.000018,0.000014,0.000020,0.000021,0.000020,0.000041,0.000014,0.000017,-2.354458e-05,0.000038
AGL.JO,4.730447e-05,0.000055,0.000047,0.000032,0.000330,4.212702e-05,0.000105,0.000032,0.000064,0.000034,...,0.000035,0.000034,0.000048,0.000023,0.000037,0.000139,0.000034,0.000023,-2.427761e-05,0.000025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAL.JO,8.590347e-05,0.000080,0.000054,0.000041,0.000139,6.155930e-05,0.000135,0.000052,0.000106,0.000035,...,0.000056,0.000047,0.000072,0.000083,0.000058,0.000469,0.000047,0.000021,-5.352153e-05,0.000079
VOD.JO,3.989491e-05,0.000041,0.000034,0.000014,0.000034,2.277270e-05,0.000034,0.000014,0.000030,0.000021,...,0.000011,0.000165,0.000021,0.000005,0.000014,0.000047,0.000269,0.000022,-3.569119e-05,0.000029
WBO.JO,2.466189e-05,0.000014,0.000047,0.000017,0.000023,2.059271e-05,0.000022,0.000010,0.000035,0.000025,...,0.000034,0.000022,0.000019,0.000015,0.000001,0.000021,0.000022,0.000229,5.312952e-06,0.000023
WEZ.JO,-4.139429e-07,0.000027,0.000059,-0.000024,-0.000024,-1.370189e-05,0.000030,0.000040,-0.000010,0.000048,...,0.000041,-0.000036,-0.000013,0.000014,0.000033,-0.000054,-0.000036,0.000005,1.255785e-03,0.000001


In [83]:
corr_matrix

,ABG.JO,ACL.JO,ADH.JO,AFE.JO,AGL.JO,AIP.JO,ANG.JO,APN.JO,ARI.JO,ARL.JO,...,TFG.JO,TKG.JO,TON.JO,TRU.JO,TSG.JO,VAL.JO,VOD.JO,WBO.JO,WEZ.JO,WHL.JO
ABG.JO,1.000000,0.363214,0.136109,0.142159,0.239156,0.381950,0.191931,0.355009,0.261461,0.199265,...,0.290712,0.235979,0.269652,0.280400,0.200575,0.342043,0.235979,0.167823,-0.000928,0.336593
ACL.JO,0.363214,1.000000,-0.052231,0.088790,0.261001,0.275987,0.321628,0.301308,0.139279,0.115387,...,0.266426,0.226452,0.218018,0.179456,0.170285,0.298726,0.226452,0.088211,0.057342,0.221520
ADH.JO,0.136109,-0.052231,1.000000,0.016415,0.163056,-0.002228,0.171823,0.047071,0.028610,0.202360,...,0.077271,0.138626,0.137787,0.117699,0.086080,0.148176,0.138626,0.221497,0.091652,0.030133
AFE.JO,0.142159,0.088790,0.016415,1.000000,0.168956,0.257192,0.101532,0.151913,0.327629,0.153147,...,0.103789,0.088384,0.109227,0.102060,0.119081,0.168858,0.088384,0.118795,-0.054548,0.203222
AGL.JO,0.239156,0.261001,0.163056,0.168956,1.000000,0.220802,0.463622,0.160443,0.224443,0.177611,...,0.169618,0.175221,0.221544,0.092927,0.189061,0.485049,0.175221,0.133977,-0.047543,0.110740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAL.JO,0.342043,0.298726,0.148176,0.168858,0.485049,0.254113,0.467439,0.204033,0.294107,0.146624,...,0.215600,0.190077,0.262240,0.268251,0.231697,1.000000,0.190077,0.098599,-0.082547,0.282013
VOD.JO,0.235979,0.226452,0.138626,0.088384,0.175221,0.139648,0.177783,0.083634,0.124104,0.128368,...,0.064415,1.000000,0.116420,0.023835,0.082295,0.190077,1.000000,0.156545,-0.081775,0.152613
WBO.JO,0.167823,0.088211,0.221497,0.118795,0.133977,0.145278,0.133098,0.066278,0.164915,0.179717,...,0.224876,0.156546,0.119922,0.085291,0.008917,0.098599,0.156545,1.000000,0.014004,0.142356
WEZ.JO,-0.000928,0.057342,0.091652,-0.054548,-0.047543,-0.031832,0.059184,0.087705,-0.016229,0.111014,...,0.088662,-0.081775,-0.027574,0.026150,0.074198,-0.082547,-0.081775,0.014004,1.000000,0.002403


In [84]:
#aus der geschrumpften Matrix die Korrelationsmatrix bauen
standardabweichung = np.sqrt(np.diag(cov_ledoit))
corr_ledoit = cov_ledoit / np.outer(standardabweichung, standardabweichung)
corr_ledoit

,ABG.JO,ACL.JO,ADH.JO,AFE.JO,AGL.JO,AIP.JO,ANG.JO,APN.JO,ARI.JO,ARL.JO,...,TFG.JO,TKG.JO,TON.JO,TRU.JO,TSG.JO,VAL.JO,VOD.JO,WBO.JO,WEZ.JO,WHL.JO
ABG.JO,1.000000,0.232593,0.094846,0.087719,0.156563,0.235427,0.125743,0.223312,0.182042,0.122715,...,0.184201,0.146240,0.173714,0.187569,0.125460,0.238607,0.146240,0.098067,-0.000702,0.218969
ACL.JO,0.232593,1.000000,-0.037294,0.056138,0.175076,0.174307,0.215908,0.194205,0.099363,0.072811,...,0.172975,0.143795,0.143913,0.123004,0.109139,0.213526,0.143795,0.052817,0.044490,0.147661
ADH.JO,0.094846,-0.037294,1.000000,0.011293,0.119021,-0.001531,0.125516,0.033014,0.022211,0.138952,...,0.054591,0.095789,0.098973,0.087787,0.060035,0.115254,0.095789,0.144316,0.077380,0.021858
AFE.JO,0.087719,0.056138,0.011293,1.000000,0.109205,0.156519,0.065675,0.094347,0.225220,0.093118,...,0.064929,0.054079,0.069474,0.067406,0.073541,0.116301,0.054079,0.068538,-0.040780,0.130530
AGL.JO,0.156563,0.175076,0.119021,0.109205,1.000000,0.142562,0.318166,0.105717,0.163690,0.114574,...,0.112578,0.113744,0.149500,0.065114,0.123874,0.354437,0.113744,0.082007,-0.037709,0.075463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VAL.JO,0.238607,0.213526,0.115254,0.116301,0.354437,0.174832,0.341829,0.143258,0.228568,0.100789,...,0.152484,0.131482,0.188571,0.200295,0.161768,1.000000,0.131482,0.064311,-0.069768,0.204782
VOD.JO,0.146240,0.143795,0.095789,0.054079,0.113744,0.085353,0.115495,0.052166,0.085681,0.078389,...,0.040472,0.614508,0.074369,0.015810,0.051043,0.131482,1.000000,0.090708,-0.061399,0.098447
WBO.JO,0.098067,0.052817,0.144316,0.068538,0.082007,0.083727,0.081531,0.038981,0.107359,0.103482,...,0.133225,0.090708,0.072234,0.053346,0.005215,0.064311,0.090708,1.000000,0.009915,0.086590
WEZ.JO,-0.000702,0.044490,0.077380,-0.040780,-0.037709,-0.023772,0.046978,0.066842,-0.013690,0.082831,...,0.068064,-0.061399,-0.021522,0.021193,0.056230,-0.069768,-0.061399,0.009915,1.000000,0.001894


In [85]:
is_in_range = np.all((corr_matrix >= -1) & (corr_matrix <= 1))

print(is_in_range)

True


In [86]:
tol = 1e-12
is_in_range_ledoit = np.all((corr_ledoit >= -1) & (corr_ledoit <= 1 + tol))

print(is_in_range_ledoit)

True


In [87]:
#ÜBERPRÜFUNG KOVARIANZ_MATRIZEN

#Alle 4 Kovarianz- und Korrelationsmatrizen haben die gleiche Größe (NxN)
#Erreicht: Ja

#Korrelationsmatrizen haben Werte zwischen -1 und 1 und haben 1 auf der Diagonale
#Erreicht: Ja

#Kovarianz und Korrelationsmatrizen unter Ledoit unterscheiden sich von den gewöhnlichen Matrizen
#Erreicht: Ja

In [88]:
gewichte_EW = equally_weighted(training_daten)
gewichte_MV = min_variance_opt(cov_matrix)
gewichte_MD = max_diversification_portfolio(cov_matrix)
gewichte_IV = inverse_volatility_portfolio(cov_matrix)
gewichte_ERC = get_risk_parity_weights(cov_matrix)
gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
gewichte_herc = herc (training_daten)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3064556389717537e-05
            Iterations: 140
            Function evaluations: 10360
            Gradient evaluations: 140
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.381970760045858
            Iterations: 60
            Function evaluations: 4491
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.854170012627015e-09
            Iterations: 41
            Function evaluations: 3035
            Gradient evaluations: 41


In [89]:
print(1/73)
print(gewichte_EW.min())
print(gewichte_EW.sum())

0.0136986301369863
0.0136986301369863
1.0


In [90]:
#ÜBERPRÜFUNG METHODE EQUALLY WEIGHTED

#Minimales Gewicht muss 1/N entsprechen
#Erreicht: Ja

#Gewichte müssen aufsummiert 1 ergeben
#Erreicht: Ja

In [91]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [92]:
#ÜBERPRÜFUNG METHODE MINIMUM-VARIANCE

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [93]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [94]:
#ÜBERPRÜFUNG METHODE MAXIMUM-DIVERSIFICATION

#Optimierer terminiert erfolgreich mit einem Zielwert unter 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [95]:
wert = gewichte_IV.min()
print(f"{wert:.12f}")
wert = gewichte_IV.sum()
print(f"{wert:.12f}")

0.004571780015
1.000000000000


In [96]:
#ÜBERPRÜFUNG METHODE INVERSE-VOLATILITY

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [97]:
wert = gewichte_ERC.min()
print(f"{wert:.12f}")
wert = gewichte_ERC.sum()
print(f"{wert:.12f}")

0.005450938913
1.000000000000


In [98]:
#ÜBERPRÜFUNG METHODE EQUAL-RISK-CONTRIBUTION

#Optimierer terminiert erfolgreich mit einem Zielwert über 0
#Erreicht: Ja

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [99]:
wert = gewichte_hrp.min()
print(f"{wert:.12f}")
wert = gewichte_hrp.sum()
print(f"{wert:.12f}")

0.001745287703
1.000000000000


In [100]:
#ÜBERPRÜFUNG METHODE HRP

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [101]:
wert = gewichte_herc.min()
print(f"{wert:.12f}")
wert = gewichte_herc.sum()
print(f"{wert:.12f}")

0.000266923979
1.000000000000


In [102]:
#ÜBERPRÜFUNG METHODE HERC

#Es gibt keine negativen Gewichte
#Erreicht: Ja

#Die Summe der Gewichte ergibt 1
#Erreicht: Ja

In [103]:
np.savetxt("test_gewichte_EW_JSE.csv", gewichte_EW, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_MV_JSE.csv", gewichte_MV, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_MD_JSE.csv", gewichte_MD, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_IV_JSE.csv", gewichte_IV, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_ERC_JSE.csv", gewichte_ERC, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_HERC_JSE.csv", gewichte_herc, delimiter=",", fmt="%.12f")
np.savetxt("test_gewichte_HRP_JSE.csv", gewichte_hrp, delimiter=",", fmt="%.12f")

In [104]:
#TEST DER ERGEBNISSE (RENDITEN AUF TEST-DATEN ANHAND GEWICHTE)

returns_EW = test_daten @ gewichte_EW
returns_MV = test_daten @ gewichte_MV
returns_MD = test_daten @ gewichte_MD
returns_IV = test_daten @ gewichte_IV
returns_ERC = test_daten @ gewichte_ERC
returns_HRP = test_daten @ gewichte_hrp
returns_HERC = test_daten @ gewichte_herc

print(returns_EW)
print(returns_MV)
print(returns_MD)
print(returns_IV)
print(returns_ERC)
print(returns_HRP)
print(returns_HERC)

Date
2007-07-02   -0.000381
2007-07-03    0.000563
2007-07-04   -0.002160
2007-07-05    0.000411
2007-07-06    0.006846
2007-07-09    0.014482
2007-07-10   -0.001892
2007-07-11   -0.003383
2007-07-12   -0.001484
2007-07-13    0.001764
2007-07-16    0.001217
2007-07-17   -0.003131
2007-07-18   -0.005792
2007-07-19    0.002187
2007-07-20    0.002967
2007-07-23    0.003922
2007-07-24   -0.005026
2007-07-25   -0.010565
2007-07-26   -0.015177
2007-07-27   -0.019921
2007-07-30    0.003056
2007-07-31    0.008836
dtype: float64
Date
2007-07-02   -0.006505
2007-07-03   -0.001707
2007-07-04    0.001233
2007-07-05    0.004777
2007-07-06    0.001687
2007-07-09    0.003285
2007-07-10   -0.004271
2007-07-11   -0.006123
2007-07-12   -0.001058
2007-07-13   -0.000569
2007-07-16   -0.002800
2007-07-17   -0.000503
2007-07-18   -0.004697
2007-07-19   -0.000188
2007-07-20    0.003626
2007-07-23    0.003284
2007-07-24   -0.004051
2007-07-25   -0.004406
2007-07-26   -0.014827
2007-07-27   -0.016192
2007-07-3

In [105]:
#ÜBERPRÜFUNG GEWICHTE UND RETURNS

#Alle Methoden geben 73 Werte (Gewichte) zurück (Array-Längen 73)
#Erreicht: Ja

#Manuell ermittelte Returns stimmen mit der Implementierung überein
#Erreicht: Ja

In [106]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_renditen_JSE = pd.read_csv(
    'returnsJSE_neu_f.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [107]:
df_renditen_JSE.info

<bound method DataFrame.info of             Date  Equally-Weighted  Minimum-Variance  Maximum-Diversification  \
0     2006-06-01          0.000033          0.000705                 0.000391   
1     2006-06-02          0.017961          0.005881                 0.008007   
2     2006-06-05          0.000017         -0.003898                -0.003364   
3     2006-06-06         -0.033173         -0.016104                -0.020517   
4     2006-06-07         -0.023522         -0.025723                -0.023729   
...          ...               ...               ...                      ...   
4905  2025-05-26          0.004559          0.002929                 0.002356   
4906  2025-05-27          0.000705          0.000401                -0.001830   
4907  2025-05-28          0.000644          0.004763                 0.005638   
4908  2025-05-29          0.008966         -0.001394                 0.001425   
4909  2025-05-30          0.003385          0.000332                 0.000241

In [108]:
#ÜBERPRÜFUNG GESAMMELTE RENDITEN

#Gesammelte Renditen starten am 01.06.2006 (da erstes Training vom 01.06.2005 bis 31.05.2006)
#Erreicht: Ja

#Gesammelte Renditen enden am 30.05.2025 (Ende der Daten)
#Erreicht: Ja

#Gesammelte Renditen sind vollständig (5170 Renditen - 260 Renditen aus erstem Jahr = 4910 berechnete Renditen)
#Erreicht: Ja

#Gesammlte Renditen aus dem Hauptprogramm stimmen mit den hier ermittelten Daten überein (im Testzeitfenster)
#Erreicht: Ja

In [109]:
#Ein einzelner Zeitraum (1 Trainingsfenster + anschließendes Testzeitfenster)
#Die Richtigkeit der Methoden soll getestet werden

#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2022-10-01')
end_datum = pd.Timestamp('2023-10-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
    
print(training_anfang)
    
#Definition des Trainingsdatensatzes und des Testdatensatzes
training_daten = returns.loc[training_anfang:training_ende]
test_daten = returns.loc[test_anfang:test_ende]
training_daten = training_daten.dropna(axis=1)
    
#Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
max_null_tage = 63
null_zaehler = (training_daten == 0).sum()
valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
training_daten = training_daten[valide_tickers]
    
#Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
test_daten = test_daten[vorhandene_assets]
test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
#Erzeugung Matrizen
cov_matrix = training_daten.cov()
corr_matrix = training_daten.corr()

2022-10-01 00:00:00


In [110]:
gewichte_EW = equally_weighted(training_daten)
gewichte_MV = min_variance_opt(cov_matrix)
gewichte_MD = max_diversification_portfolio(cov_matrix)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1798537812387731e-05
            Iterations: 249
            Function evaluations: 38844
            Gradient evaluations: 249
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.396765432658553
            Iterations: 114
            Function evaluations: 17382
            Gradient evaluations: 110


In [111]:
wert = gewichte_MV.min()
print(f"{wert:.12f}")
wert = gewichte_MV.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000000


In [112]:
wert = gewichte_MD.min()
print(f"{wert:.12f}")
wert = gewichte_MD.sum()
print(f"{wert:.12f}")

0.000000000000
1.000000000023


In [113]:
returns_EW = test_daten @ gewichte_EW
returns_MV = test_daten @ gewichte_MV
returns_MD = test_daten @ gewichte_MD

print(returns_EW)
print(returns_MV)
print(returns_MD)

Date
2023-10-02   -0.011239
2023-10-03   -0.008195
2023-10-04   -0.005397
2023-10-05    0.003899
2023-10-06    0.005699
2023-10-09   -0.003541
2023-10-10    0.014428
2023-10-11    0.000113
2023-10-12    0.000020
2023-10-13   -0.005972
2023-10-16    0.001968
2023-10-17   -0.001801
2023-10-18   -0.008920
2023-10-19   -0.007068
2023-10-20   -0.007741
2023-10-23    0.002652
2023-10-24   -0.000937
2023-10-25   -0.001916
2023-10-26   -0.006368
2023-10-27   -0.002415
2023-10-30   -0.000078
dtype: float64
Date
2023-10-02    0.001763
2023-10-03   -0.006470
2023-10-04   -0.004098
2023-10-05    0.003333
2023-10-06    0.004015
2023-10-09   -0.009286
2023-10-10    0.000984
2023-10-11    0.003691
2023-10-12   -0.000981
2023-10-13   -0.005274
2023-10-16    0.000755
2023-10-17   -0.002506
2023-10-18   -0.002157
2023-10-19   -0.001327
2023-10-20   -0.007889
2023-10-23   -0.002625
2023-10-24    0.001165
2023-10-25   -0.002466
2023-10-26   -0.004515
2023-10-27   -0.000686
2023-10-30    0.003884
dtype: fl